<a href="https://colab.research.google.com/github/nin-ed/CNN-Based-Algorithms/blob/main/Style%20Transfer/Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torch import optim, nn
from torchvision import transforms, models
from torchvision.utils import save_image

In [52]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

In [58]:
def load_image(img, size):
    img = Image.open(img).convert('RGB')
    """transform = transforms.Compose([transforms.Resize(size), transforms.ToTensor(),
                                    transforms.Normalize(mean, std)])"""
    transform = transforms.Compose([transforms.Resize(size), transforms.ToTensor()])
    img = transform(img)[:3, :, :].unsqueeze(0)
    return img.cuda()


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = [0, 5, 10, 19, 28]
        self.vgg = models.vgg19(pretrained=True).features[:29]
    
    def forward(self, x):
        feat = []
        for n, l in enumerate(self.vgg):
            x = l(x)
            if n in self.layers:
                feat.append(x)
        return feat

In [59]:
def content_loss(orig, gen):
    loss = torch.mean((gen-orig)**2)
    return loss


def style_loss(style, gen):
    _, ch, h, w = gen.shape
    G = torch.mm(gen.view(ch, h*w), gen.view(ch, h*w).t())
    A = torch.mm(style.view(ch, h*w), style.view(ch, h*w).t())
    loss = torch.mean((G-A)**2)
    return loss


def total_loss(orig, gen, style, alpha, beta):
    c_loss, s_loss = 0, 0
    for o, g, s in zip(orig, gen, style):
        c_loss += content_loss(o, g)
        s_loss += style_loss(s, g)

    loss = alpha*c_loss + beta*s_loss
    return loss

In [60]:
orig = load_image("/content/drive/MyDrive/elon.jpg", (400, 400))
style = load_image("/content/drive/MyDrive/painting.jpeg", (400, 400))
gen = orig.clone().requires_grad_(True)

model = Net()
model = model.cuda()

optimizer = optim.Adam([gen], lr=0.001)

In [61]:
# epochs = 1000
epochs = 5000

for e in range(1, epochs+1):
    optimizer.zero_grad()
    orig_f = model(orig)
    gen_f = model(gen)
    style_f = model(style)
    loss = total_loss(orig_f, gen_f, style_f, 8, 70)
    loss.backward()
    optimizer.step()
    if e%100 == 0:
        print(f"Epoch: {e}/{epochs} ----- Loss: {loss}")

Epoch: 100/5000 ----- Loss: 340674464.0
Epoch: 200/5000 ----- Loss: 190711616.0
Epoch: 300/5000 ----- Loss: 129543328.0
Epoch: 400/5000 ----- Loss: 92197520.0
Epoch: 500/5000 ----- Loss: 68308768.0
Epoch: 600/5000 ----- Loss: 53612144.0
Epoch: 700/5000 ----- Loss: 44953228.0
Epoch: 800/5000 ----- Loss: 39816036.0
Epoch: 900/5000 ----- Loss: 36487272.0
Epoch: 1000/5000 ----- Loss: 34048548.0
Epoch: 1100/5000 ----- Loss: 32068578.0
Epoch: 1200/5000 ----- Loss: 30361578.0
Epoch: 1300/5000 ----- Loss: 28834100.0
Epoch: 1400/5000 ----- Loss: 27439322.0
Epoch: 1500/5000 ----- Loss: 26154044.0
Epoch: 1600/5000 ----- Loss: 24964186.0
Epoch: 1700/5000 ----- Loss: 23852160.0
Epoch: 1800/5000 ----- Loss: 22802932.0
Epoch: 1900/5000 ----- Loss: 21808264.0
Epoch: 2000/5000 ----- Loss: 20862132.0
Epoch: 2100/5000 ----- Loss: 19960242.0
Epoch: 2200/5000 ----- Loss: 19097154.0
Epoch: 2300/5000 ----- Loss: 18269216.0
Epoch: 2400/5000 ----- Loss: 17473028.0
Epoch: 2500/5000 ----- Loss: 16707001.0
Epoch:

In [62]:
save_image(gen, "/content/sample_data/elon1.png")